## Procedure

1. Collect and clean data (NASA Exoplanet Archive)
2. 

In [2]:
# imports
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import random 
import pandas as pd
from random import sample

# Setting up dataframe
exoplanetdata = pd.read_csv('exoplanetdata.csv', skiprows = 17) 
exoplanetdata['st_lum'] = exoplanetdata['st_lum'].fillna(0) # Changing nan to 0 for eaiser indexing
exoplanetdata['pl_orbsmax'] = exoplanetdata['pl_orbsmax'].fillna(0) # Changing nan to 0 for eaiser indexing
exoplanetdata['pl_radj'] = exoplanetdata['pl_radj'].fillna(0) # Changing nan to 0 for eaiser indexing
exoplanetdata['st_rad'] = exoplanetdata['st_rad'].fillna(1) # Changing nan to 1 for eaiser indexing
exoplanetdata['st_teff'] = exoplanetdata['st_teff'].fillna(4920) # Changing nan to 4920 K for eaiser indexing
exoplanetdata['pl_orbper'] = exoplanetdata['pl_orbper'].fillna(50) # Changing nan to 4920 K for eaiser indexing

exoplanetdata = exoplanetdata.set_index('pl_hostname') 


# INPUT NECESSARY VALUES
#print('Input a star with known orbiting exoplanets. If you would like to explore a Kepler Star, input Kepler-(a value in') 
#print('most values in the range 4 to 1661) ex: Kepler-4. Few values in range not included, but most are. If not interested') 
#print('in Kepler, input a differently named host star.')
Star_name = input()
#Star_name = exoplanetdata.sample(1)
#Star_name = Star_name.index[0]
Star_name = ('{}'.format(Star_name))
# Locate star in dataframe
Kepler_star_data = exoplanetdata.loc[[Star_name]]


# Star Luminosity - Better way to do this??/ maybe not with current dataframe
# if originally no exact luminosity is presented in data seat, we can make a good estimate with L = R^2*T^4
Star_Luminosity = []

if Kepler_star_data['st_lum'].iloc[0] == 0: 
    Star_Luminosity.append((Kepler_star_data['st_rad'].iloc[0]**2)*((Kepler_star_data['st_teff'].iloc[0]/5778)**4))

else:
    Star_Luminosity.append(10**(Kepler_star_data['st_lum'].iloc[0])) 

Star_Luminosity = str(Star_Luminosity)
Star_Luminosity =  Star_Luminosity.strip('[').strip(']')
Star_Luminosity = round(float(Star_Luminosity),3)
print(Star_name,'has a luminosity of',Star_Luminosity, 'Solar units')

# star size (plot units; to show relative size of stars)
Star_size = []

if Kepler_star_data['st_rad'].iloc[0] >= 0 and Kepler_star_data['st_rad'].iloc[0] <= 1:
    Star_size.append(0.05)
elif Kepler_star_data['st_rad'].iloc[0] > 1 and Kepler_star_data['st_rad'].iloc[0] <= 2:
    Star_size.append(0.2)
elif Kepler_star_data['st_rad'].iloc[0] > 2 and Kepler_star_data['st_rad'].iloc[0] <= 5:
    Star_size.append(0.5)
elif Kepler_star_data['st_rad'].iloc[0] > 5 and Kepler_star_data['st_rad'].iloc[0] <= 10:
    Star_size.append(0.75)
elif Kepler_star_data['st_rad'].iloc[0] > 10:
    Star_size.append(1)

Star_size = str(Star_size)
Star_size =  Star_size.strip('[').strip(']')
Star_size = float(Star_size)
print('Star size is', Kepler_star_data['st_rad'].iloc[0],'Solar units')
print('Star size is',Star_size,'map units')

# Number of exoplanets - not needed for now
#number_exo = Kepler_star_data['pl_pnum'].iloc[0]

# Exoplanet's semi-major axes
planet_axes_list = []
for i in range(len(Kepler_star_data['pl_orbsmax'])):
    if Kepler_star_data['pl_orbsmax'].iloc[i] == 0:
        planet_axes_list.append(round(((Kepler_star_data['pl_orbper'].iloc[i]/(365))**2)**(1/3),3)) # Based off T^2 = a^3 - nice approx. for now 
    
    else:
        planet_axes_list.append(round(Kepler_star_data['pl_orbsmax'].iloc[i],3))
planet_axes_list = sorted(planet_axes_list, key = lambda x:float(x))



# Habitable zone function
# Limits 0.9 and 1.5 are sourced from britannica.com (https://www.britannica.com/science/habitable-zone)
def habit_zone(L):
    # L input is given star's luminosity (in Solar units)
    Lsun = 1.0 # Luminosity of Sun (in Solar units)
    d_inner = round(0.9*(np.sqrt(L/Lsun)),3)
    #print('Inner limit is',d_inner,'AU')
    d_outer = round(1.5*(np.sqrt(L/Lsun)),3)
    #print('Outer limit is',d_outer,'AU')

    return d_inner, d_outer    

inner_limit = habit_zone(Star_Luminosity)[0]
outer_limit = habit_zone(Star_Luminosity)[1]

print('The inner limit is',inner_limit,' AU; the outer limit is', outer_limit, 'AU')

# ZONE HABITABILITY (Blue, Green, Red)

theta = np.linspace(0., 2.*np.pi, 100, endpoint=True) # For circle

x_red = inner_limit*np.cos(theta) # Red zone
y_red = inner_limit*np.sin(theta)

x_green = outer_limit*np.cos(theta) # Green zone
y_green = outer_limit*np.sin(theta)

x_blue = (outer_limit+0.5)*np.cos(theta) # Blue zone
y_blue = (outer_limit+0.5)*np.sin(theta) 

xsun = (Star_size)*np.cos(theta) # Sun 
ysun = (Star_size)*np.sin(theta)

# PLOTTING

fig, ax = plt.subplots(figsize=(20, 20)) 

#set the limits of the figure (based on last inputted radii)
ax.set_xlim(-outer_limit-0.6, outer_limit+0.6) 
ax.set_ylim(-outer_limit-0.6, outer_limit+0.6) 

# Fill in habitability zones based off limits 
ax.fill_between(x_blue, y_blue, -y_blue, color='blue',alpha=0.8)
ax.fill_between(x_green, y_green, -y_green, color='green',alpha=0.8)
ax.fill_between(x_red, y_red, -y_red, color='red',alpha=0.8)
ax.fill_between(xsun, ysun, -ysun, color='yellow')

# Creating data labels
planet_index = ['b','c','d','e','f','g','h','i','j','k']
planet_index_rows = []
for i in range(len(planet_axes_list)):
    index = Star_name + planet_index[i]
    planet_index_rows.append(index)

# Setting up orbits
Circle_list = []
for i in range(len(planet_axes_list)):
    if planet_axes_list[i] > Star_size: 
        Circle_index = plt.Circle((0,0), planet_axes_list[i] , color='black', fill=False,alpha = 2)
        Circle_list.append(Circle_index)
    elif planet_axes_list[i] <= Star_size:
        Circle_index = plt.Circle((0,0), planet_axes_list[i] + (xsun[0]) , color='black', fill=False,alpha = 2)
        Circle_list.append(Circle_index)
        

# Display orbits
for i in Circle_list:
    ax.add_artist(i)
    
# Add planet (circle) marker
colors_marker = ['bo','ro','go','co', 'mo', 'yo', 'ko','mo','co','yo'] # color of planet
colors_marker = sample(colors_marker, len(planet_axes_list))
for i in range(len(planet_axes_list)):
    if planet_axes_list[i] > Star_size and Kepler_star_data['pl_radj'].iloc[i] < 0.5:
        plt.plot(planet_axes_list[i],0,colors_marker[i],ms = 10, label = (planet_axes_list[i],planet_index_rows[i]), alpha = 1)
    
    elif planet_axes_list[i] > Star_size and Kepler_star_data['pl_radj'].iloc[i] >= 0.5 and Kepler_star_data['pl_radj'].iloc[i] < 1.5:
        plt.plot(-planet_axes_list[i],0,colors_marker[i],ms = 20, label = (planet_axes_list[i],planet_index_rows[i]), alpha = 1)
    
    elif planet_axes_list[i] > Star_size and Kepler_star_data['pl_radj'].iloc[i] >= 1.5:
        plt.plot(-planet_axes_list[i],0,colors_marker[i],ms = 30, label = (planet_axes_list[i],planet_index_rows[i]), alpha = 1)   
    
    elif planet_axes_list[i] <= Star_size and Kepler_star_data['pl_radj'].iloc[i] < 0.5:
        plt.plot(planet_axes_list[i]+(Star_size),0,colors_marker[i],ms = 10, label = (planet_axes_list[i],planet_index_rows[i]), alpha = 1)
    
    elif planet_axes_list[i] <= Star_size and Kepler_star_data['pl_radj'].iloc[i] >= 0.5 and Kepler_star_data['pl_radj'].iloc[i] < 1.5:
        plt.plot(-planet_axes_list[i]-(Star_size),0,colors_marker[i],ms = 20, label = (planet_axes_list[i],planet_index_rows[i]), alpha = 1)
    
    elif planet_axes_list[i] <= Star_size and Kepler_star_data['pl_radj'].iloc[i] >= 1.5:
        plt.plot(-planet_axes_list[i]-(Star_size),0,colors_marker[i],ms = 30, label = (planet_axes_list[i],planet_index_rows[i]), alpha = 1)

plt.legend(title="Semi-major axis (AU)",prop={'size': 15})
plt.title(Star_name, fontsize=30)
print('The exoplanets inlcude', planet_index_rows)


# ------------------------------------------------------------------------------------------------------------------
# Check if planets are habitable (temperature; 273 K to 373 K)
# We can approximate planet surface temperature with the equation: T_planet = T_star*(R_star/2*semi-major)**(1/2)*(1-A)**(1/4)
# A is the planets albedo - good estimate is 0.35
# We multiply by 215.032 because this is the factor to convert AU to Solar radius 

Planet_temp = []
A = 0.3 # albedo
for i in planet_axes_list:
    planet_temp = Kepler_star_data['st_teff'].iloc[0]*((Kepler_star_data['st_rad'].iloc[0]/(2*i*215.032))**(1/2)*(1-A)**(1/4))
    Planet_temp.append(round(planet_temp,2))

# If planet is within 0 and 100 Celsius and within the 'green' zone
Yes_No_list = []
for i in range(len(Planet_temp)):
    if Planet_temp[i] >= 273 and Planet_temp[i] <= 373 and planet_axes_list[i] >= inner_limit and planet_axes_list[i] <= outer_limit:
        Yes = 'Yes'
        Yes_No_list.append(Yes)
    elif Planet_temp[i] >= 200 and Planet_temp[i] <=450 and planet_axes_list[i] >= inner_limit and planet_axes_list[i] <= outer_limit:
        Maybe = 'Likely'
        Yes_No_list.append(Maybe)
    else:
        No = 'No'
        Yes_No_list.append(No)
        
Yes_No_list_Zone = []
for i in range(len(planet_axes_list)):
    if planet_axes_list[i] >= inner_limit and planet_axes_list[i] <= outer_limit:
        Yes = 'Yes'
        Yes_No_list_Zone.append(Yes)
    elif planet_axes_list[i] <= inner_limit or planet_axes_list[i] >= outer_limit:
        No = 'No'
        Yes_No_list_Zone.append(No)

Jupyter_list = []
for i in range(len(planet_axes_list)):
    if Kepler_star_data['pl_radj'].iloc[i] < 0.5:
        Smaller = 'Smaller'
        Jupyter_list.append(Smaller)
    elif Kepler_star_data['pl_radj'].iloc[i] >= 0.5 and Kepler_star_data['pl_radj'].iloc[i] < 1.5:
        Yes = 'Yes'
        Jupyter_list.append(Yes)
    elif Kepler_star_data['pl_radj'].iloc[i] >= 1.5:
        Larger = 'Larger'
        Jupyter_list.append(Larger)
    

# DataFrame of habitability check        
Habitability = pd.DataFrame(data = list(zip(Planet_temp,Jupyter_list,Yes_No_list_Zone,Yes_No_list)),index = planet_index_rows, columns = ['Surface Temperature (K)','Jupyter-like?', 'Within Habitable Zone?','Habitable?'])
display(Habitability)

plt.show()



KeyError: 'st_lum'